# Capstone Project 

# Load the training data

In [1]:
import pandas as pd

df = pd.read_csv("train.csv")

# See the shape of our data

In [2]:
df.shape

(5634, 22)

# List Column Names

In [3]:
df.columns

Index(['ID', 'customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
       'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
       'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
       'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

# Inspect the data we have

In [4]:
df.isnull().sum()

ID                  0
customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [5]:
df.gender.value_counts()

Male      2833
Female    2801
Name: gender, dtype: int64

In [6]:
df.SeniorCitizen.value_counts()

0    4714
1     920
Name: SeniorCitizen, dtype: int64

In [7]:
df.Partner.value_counts()

No     2905
Yes    2729
Name: Partner, dtype: int64

In [8]:
df.Dependents.value_counts()

No     3955
Yes    1679
Name: Dependents, dtype: int64

In [9]:
df.tenure.value_counts()

1     487
72    292
2     187
3     150
71    142
     ... 
21     45
44     43
28     42
36     40
0       8
Name: tenure, Length: 73, dtype: int64

In [10]:
df.PhoneService.value_counts()

Yes    5075
No      559
Name: PhoneService, dtype: int64

In [11]:
df.MultipleLines.value_counts()

No                  2685
Yes                 2390
No phone service     559
Name: MultipleLines, dtype: int64

In [12]:
df.InternetService.value_counts()

Fiber optic    2483
DSL            1937
No             1214
Name: InternetService, dtype: int64

In [13]:
df.OnlineSecurity.value_counts()

No                     2797
Yes                    1623
No internet service    1214
Name: OnlineSecurity, dtype: int64

In [14]:
df.OnlineBackup.value_counts()

No                     2442
Yes                    1978
No internet service    1214
Name: OnlineBackup, dtype: int64

In [15]:
df.DeviceProtection.value_counts()

No                     2472
Yes                    1948
No internet service    1214
Name: DeviceProtection, dtype: int64

In [16]:
df.TechSupport.value_counts()

No                     2771
Yes                    1649
No internet service    1214
Name: TechSupport, dtype: int64

In [17]:
df.StreamingTV.value_counts()

No                     2226
Yes                    2194
No internet service    1214
Name: StreamingTV, dtype: int64

In [18]:
df.StreamingMovies.value_counts()

No                     2217
Yes                    2203
No internet service    1214
Name: StreamingMovies, dtype: int64

In [19]:
df.Contract.value_counts()

Month-to-month    3102
Two year          1359
One year          1173
Name: Contract, dtype: int64

In [20]:
df.PaperlessBilling.value_counts()

Yes    3331
No     2303
Name: PaperlessBilling, dtype: int64

In [21]:
df.PaymentMethod.value_counts()

Electronic check             1891
Mailed check                 1286
Bank transfer (automatic)    1244
Credit card (automatic)      1213
Name: PaymentMethod, dtype: int64

In [22]:
df.MonthlyCharges.value_counts()

20.05     50
20.00     37
19.65     37
19.85     36
19.95     36
          ..
107.60     1
63.25      1
71.45      1
71.95      1
102.80     1
Name: MonthlyCharges, Length: 1489, dtype: int64

In [23]:
df.TotalCharges.value_counts()

20.2       9
           8
45.3       7
19.65      7
19.75      6
          ..
2802.3     1
8100.55    1
8547.15    1
4812.75    1
109.3      1
Name: TotalCharges, Length: 5276, dtype: int64

In [24]:
df.Churn.value_counts()

0    4139
1    1495
Name: Churn, dtype: int64

# Compute median of TotalCharges
We learned by applying models to our data that we had non-numeric data in our feature set. Debugging with print statements, we were able to narrow the problem down to the TotalCharges column which contains mostly numeric data; however, it contains spaces in 8 rows. We will replace missing values with this median in our pipeline.

In [25]:
pd.to_numeric(df.TotalCharges, errors='coerce').median()

1398.125

# Make function for pre model fitting
We do the following fixes to the feature set:

- Fill in missing data in TotalCharges column
- Delete customerID
- Delete PhoneService. PhoneService is a summarised version of MultipleLines
- MultipleLines - Yes=2; No=1; No phone Service=0. The reason for this is that this is an ordinal value. 2 is the most phone service while 0 is the least.
- Contract - This is also an ordinal value. Month-to-month=0; One year=1; Two year=2. The higher the value, the more obligation.
- PaymentMethod - Here we remove whitepace and special characters.
- For OnlineSecurity, OnlineBackup, DeviceProtection, TechSupport, StreamingTV, and StreamingMovies: 1=Yes, 0=Anything else
- InternetService - FiberOptic=2; DSL=1; None=0. This is an ordinal value the higher the number, the faster the internet service.

Our with_all_fixes() function takes the original feature set and returns the fixed up feature set, leaving the original feature set unchanged.

In [26]:
def fix_multiple_lines(df):

    def fix(x):
        if x == 'Yes':
            return 2
        elif x == 'No':
            return 1
        else:
            return 0

    df['MultipleLines'] = df['MultipleLines'].transform(fix)


def fix_internet_service(df):

    def fix(x):
        if x == 'Fiber optic':
            return 2
        elif x == 'DSL':
            return 1
        else:
            return 0

    df['InternetService'] = df['InternetService'].transform(fix)


def fix_internet_feature(df, col_name):

    def fix(x):
        if x == 'Yes':
            return 1
        else:
            return 0

    df[col_name] = df[col_name].transform(fix)
    

def fix_contract(df):

    def fix(x):
        if x == 'Two year':
            return 2
        elif x == 'One year':
            return 1
        else:
            return 0

    df['Contract'] = df['Contract'].transform(fix)


def fix_payment_method(df):

    def fix(x):
        if x == 'Electronic check':
            return 'electronic_check'
        elif x == 'Mailed check':
            return 'mailed_check'
        elif x.startswith('Bank transfer'):
            return 'bank_transfer'
        else:
            return 'credit_card'

    df['PaymentMethod'] = df['PaymentMethod'].transform(fix)
 

def with_all_fixes(df):
    df = df.copy()

    # Replace missing TotalCharges with NaN
    df.TotalCharges = pd.to_numeric(df.TotalCharges, errors='coerce')

    # Fill NaNs with median of 1398.125
    df.TotalCharges.fillna(1398.125, inplace=True)
    
    del df['customerID']
    del df['PhoneService']
    fix_multiple_lines(df)
    fix_internet_service(df)
    fix_internet_feature(df, 'OnlineSecurity')
    fix_internet_feature(df, 'OnlineBackup')
    fix_internet_feature(df, 'DeviceProtection')
    fix_internet_feature(df, 'TechSupport')
    fix_internet_feature(df, 'StreamingTV')
    fix_internet_feature(df, 'StreamingMovies')
    fix_contract(df)
    fix_payment_method(df)
    
    one_hot_columns = [
        'gender',
        'Partner',
        'Dependents',
        'PaperlessBilling',
        'PaymentMethod',
    ]

    return pd.get_dummies(df, columns=one_hot_columns, drop_first=True)

# Fix up training data
The target column is 'Churn' and the original feature set is everything but the ID and Churn columns (the first and last columns).

In [27]:
y = df.loc[:, 'Churn'].values
X = with_all_fixes(df.iloc[:,1:-1]).values

# Training and Test data set
We use 30% for testing because we have a smaller dataset

In [28]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=1, stratify=y)

# Feature Scaling

In [29]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

# Logistic Regression

In [30]:
from sklearn.metrics import roc_curve, auc
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

pipe_lr = make_pipeline(
    StandardScaler(),
    PCA(n_components=5),
    LogisticRegression(penalty='l2', random_state=1, solver='lbfgs', C=1.0),
)

probas = pipe_lr.fit(X_train, y_train).predict_proba(X_test)

fpr, tpr, thresholds = roc_curve(y_test, probas[:, 1], pos_label=1)
print(auc(fpr, tpr))

0.8086999558869415


# Read test.csv

In [31]:
testdf = pd.read_csv('test.csv')

# Save off ID columns for submission file
ids = testdf.ID.values

# fix up the data the same way we did for our training set
X_testdata = with_all_fixes(testdf.iloc[:,1:]).values


# Apply the model

In [32]:
y_predict = pipe_lr.predict(X_testdata)


## Write final CSV

In [33]:
final = pd.DataFrame({'ID': ids, 'Churn': y_predict})
final.to_csv('submission.csv', index=False)